**UPDATION OF PATIENT'S RECORDS**

In [0]:
%sql
-- STEP 1: CLOSE OLD RECORD (UPDATE)
MERGE INTO gold_patient_scd2_dtable AS t
USING silver_patient_dtable AS s
ON t.PRIMARY_KEY = s.PRIMARY_KEY
AND t.CURR_RCD_IND = 1

WHEN MATCHED AND t.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN
UPDATE SET
    t.CURR_RCD_IND = 0,
    t.TransactionCode = 'U',
    t.UPDT_TS = current_timestamp()
;


In [0]:
%sql
select * from gold_patient_scd2_dtable where TransactionCode = 'U'

In [0]:
%sql
-- STEP 2: INSERT NEW + UPDATED ACTIVE RECORDS
INSERT INTO gold_patient_scd2_dtable
SELECT
    s.PATIENTID,
    s.FIRST,
    s.LAST,
    s.FULL_NAME,
    s.BIRTHDATE,
    s.DEATHDATE,
    s.MARITAL,
    s.GENDER,
    s.ADDRESS,
    s.CITY,
    s.STATE,
    s.COUNTRY,
    s.HOSPITALNAME,
    s.REASONDESCRIPTION,
    s.PAYERID,
    s.PRIMARY_KEY,
    s.CHK_SUM_TXT,
    CASE 
        WHEN g.CHK_SUM_TXT IS NULL THEN 'I'   -- brand new
        WHEN g.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN 'I'  -- new version
    END AS TransactionCode,
    1 AS CURR_RCD_IND,
    s.ActivationInd,
    current_timestamp() AS CRET_TS,
    TIMESTAMP('9999-12-31') AS UPDT_TS,
    s.LOAD_CTL_KEY
FROM silver_patient_dtable s
LEFT JOIN gold_patient_scd2_dtable g
    ON s.PRIMARY_KEY = g.PRIMARY_KEY 
    --AND g.CURR_RCD_IND = 1
WHERE 
      g.CHK_SUM_TXT IS NULL            -- new
   OR g.CHK_SUM_TXT <> s.CHK_SUM_TXT;  -- changed


In [0]:
%sql
select * from gold_patient_scd2_dtable

**DELETION OF INACTIVE PATIENT RECORDS**
### SOFT DELETE

In [0]:
%sql
-- STEP 0: SOFT DELETE ONLY INACTIVE RECORDS FROM SOURCE
MERGE INTO gold_patient_scd2_dtable AS t
USING silver_patient_dtable AS s
ON t.PRIMARY_KEY = s.PRIMARY_KEY
  --AND t.CURR_RCD_IND = 1          -- Only active records in GOLD
WHEN MATCHED 
    AND s.ActivationInd = 'Inactive'   -- Only if source marks it INACTIVE
THEN 
  UPDATE SET
       -- t.CURR_RCD_IND = 0,
        --t.ActivationInd = 'Inactive',
        t.TransactionCode = 'D',   -- D = Deleted / Inactive
        t.UPDT_TS = current_timestamp();


In [0]:
%sql
select * from gold_patient_scd2_dtable where TransactionCode = 'D'

**HARD DELETE**

In [0]:
%sql
--DELETE FROM gold_patient_scd2_dtable
-- WHERE TransactionCode = 'D';

**UPDATION OF PAYER'S RECORDS**

In [0]:
%sql
-- STEP 1: CLOSE OLD RECORD (UPDATE)
MERGE INTO gold_payer_scd2_dtable AS t
USING silver_payer_dtable AS s
ON t.PRIMARY_KEY = s.PRIMARY_KEY
AND t.CURR_RCD_IND = 1

WHEN MATCHED AND t.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN
UPDATE SET
    t.CURR_RCD_IND = 0,
    t.TransactionCode = 'U',
    t.UPDT_TS = current_timestamp()
;


In [0]:
%sql
select * from gold_payer_scd2_dtable where TransactionCode = 'U'

In [0]:
%sql
-- STEP 2: INSERT NEW + UPDATED ACTIVE RECORDS
INSERT INTO gold_payer_scd2_dtable
SELECT
    s.PAYERID,
    s.PAYERID,
    s.NAME,
    s.ADDRESS,
    s.CITY,
    s.STATE_HEADQUARTERED,
    s.ZIP,
    s.PHONE,
    s.PRIMARY_KEY,
    s.CHK_SUM_TXT,
    CASE 
        WHEN g.CHK_SUM_TXT IS NULL THEN 'I'   -- brand new
        WHEN g.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN 'I'  -- new version
    END AS TransactionCode,
    1 AS CURR_RCD_IND,
    s.ActivationInd,
    current_timestamp() AS CRET_TS,
    TIMESTAMP('9999-12-31') AS UPDT_TS,
    s.LOAD_CTL_KEY
FROM silver_payer_dtable s
LEFT JOIN gold_payer_scd2_dtable g
    ON s.PRIMARY_KEY = g.PRIMARY_KEY 
    --AND g.CURR_RCD_IND = 1
WHERE 
      g.CHK_SUM_TXT IS NULL            -- new
   OR g.CHK_SUM_TXT <> s.CHK_SUM_TXT;  -- changed


### **DELETION OF INACTIVE PAYER RECORDS

SOFT DELETE**

In [0]:
%sql
-- STEP 0: SOFT DELETE ONLY INACTIVE RECORDS FROM SOURCE
MERGE INTO gold_payer_scd2_dtable AS t
USING silver_payer_dtable AS s
ON t.PRIMARY_KEY = s.PRIMARY_KEY
   --AND t.CURR_RCD_IND = 1          -- Only active records in GOLD

WHEN MATCHED 
     AND s.ActivationInd = 'Inactive'   -- Only if source marks it INACTIVE
THEN 
  UPDATE SET
        --t.CURR_RCD_IND = 0,
        --t.ActivationInd = 'Inactive',
        t.TransactionCode = 'D',   -- D = Deleted / Inactive
        t.UPDT_TS = current_timestamp();


In [0]:
%sql
select * from gold_payer_scd2_dtable where TransactionCode = 'D'

**HARD DELETE**

In [0]:
%sql
--DELETE FROM gold_payer_scd2_dtable
--WHERE TransactionCode = 'D';

In [0]:
%sql
select * from gold_payer_scd2_dtable